In [ ]:
# Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import glob
import os

import librosa
import librosa.display


import torch
from torch import nn
from torch.nn import Linear
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv


In [ ]:
# GTZAN dataset
general_path = 'Data'
print(list(os.listdir(f'{general_path}/genres_original/')))

In [ ]:
# Next Steps
# Ensure all MEL-spectrograms are same NxN dimensions
# Convert them into compressed vector representations
# create train-test-split,

In [ ]:
df = pd.read_csv("Data/features_3_sec.csv")
df.head()

In [ ]:
df.shape

In [ ]:
'''
Some logic derived from reading through the following Kaggle set: https://www.kaggle.com/code/aishwarya2210/let-s-tune-the-music-with-cnn-xgboost/notebook
This is for scaling and extracting the features as needed, and converting the labels to numerical values. 
This will help to create the training and testing sets, then will be fed into the model
'''

# Label Encoding - encod the categorical classes with numerical integer values for training

# Blues - 0
# Classical - 1
# Country - 2
# Disco - 3
# Hip-hop - 4 
# Jazz - 5  
# Metal - 6 
# Pop - 7
# Reggae - 8
# Rock - 9

#To convert categorical data into model-understandable numerica data
class_list = df.iloc[:, -1]
convertor = LabelEncoder()

df = df.drop(labels="filename", axis=1)
df.head()

In [ ]:
#Fitting the label encoder & return encoded labels
y = convertor.fit_transform(class_list)
y

In [ ]:

fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:, :-1], dtype = float))

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df.iloc[:,:-1],y,test_size=0.3)
x_train.head()
# y_train.item(1)
x_train.shape

In [ ]:
# IGNORING PREVIOUSLY MADE TRAIN/TEST SPLIT

# Removing other unecessary variables
df = df.drop(labels="length", axis=1)
df = df.drop(labels="label", axis=1)

# Creataing edge tensor, 1 - Identity Matrix --> tensor
adj_matrix = 1 - torch.eye(df.shape[0])
edge_index = torch.nonzero(adj_matrix, as_tuple=False).t()

# Setting data features to tensor
x = torch.tensor(df.values)
x = x.to(torch.float32)

# Setting y to tensor
y = torch.tensor(y, dtype=torch.long)  # Make sure target is of type long (integer labels)

# Create data object
data = Data(x=x, edge_index=edge_index, y=y)

# Set train/test split
train_mask = torch.rand(df.shape[0]) < 0.80  # 80% training nodes
test_mask = ~train_mask                     # 20% testing nodes (the inverse of the train_mask)
data.train_mask = train_mask
data.test_mask = test_mask

# Check dimmensions
data

In [ ]:
# Modified from https://www.geeksforgeeks.org/graph-neural-networks-with-pytorch/

class CustomGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CustomGNN, self).__init__()
        self.layer1 = GCNConv(input_dim, hidden_dim)
        self.layer2 = GCNConv(hidden_dim, output_dim)

    def forward(self, feature_data, edge_info):
        # First GCN layer
        x = self.layer1(feature_data, edge_info)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # Second GCN layer
        x = self.layer2(x, edge_info)
        return F.log_softmax(x, dim=1)

# Initialize the GNN model
input_features = df.shape[1]
hidden_dim = df.shape[1] # PICK!
num_classes = 10
learning_rate = 0.001
weight_decay=5e-4

model = CustomGNN(input_features, hidden_dim, num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

graph_data = data  # Get the graph data

def train_model():
    model.train()
    optimizer.zero_grad()
    output = model(graph_data.x, graph_data.edge_index)
    loss = F.nll_loss(output[graph_data.train_mask], graph_data.y[graph_data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss_value = train_model()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch+1:03d}, Loss: {loss_value:.4f}')

def evaluate_model():
    model.eval()
    with torch.no_grad():
        predictions = model(graph_data.x, graph_data.edge_index).argmax(dim=1)
        correct = (predictions[graph_data.test_mask] == graph_data.y[graph_data.test_mask]).sum()
        acc = int(correct) / int(graph_data.test_mask.sum())
    return acc

accuracy = evaluate_model()
print(f'Test Accuracy: {accuracy:.4f}')